# Interactive Development Workflow


In [3]:
import sys
sys.path.insert(0, '..')

!pip install -r "/Users/ozkanuysal/Desktop/test2/test/requirements.txt"
from src.pipeline.model import create_model
from src.pipeline.data_loader import DataLoaderFactory
from src.resources.resource_pool import ResourcePoolManager, PoolType
from src.resources.gpu_manager import get_gpu_manager

print("✓ Setup complete!")

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Using cached urllib3-1.26.20-py2.py3-none-any.whl.metadata (50 kB)
INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This 

## 1. Create Development Resource Pool

In [4]:
# Get GPU manager
gpu_manager = get_gpu_manager()

# Create pool manager
pool_manager = ResourcePoolManager(gpu_manager)

# Create development pool (allows preemption)
if gpu_manager.num_gpus > 0:
    dev_pool = pool_manager.create_pool(
        name="interactive_dev",
        pool_type=PoolType.DEVELOPMENT,
        gpu_ids=[0],  # Use first GPU
        max_job_duration=3600,  # 1 hour max
        allow_preemption=True
    )
    
    print(f"Development pool created: {dev_pool.get_stats()}")
else:
    print("No GPUs available - running in CPU mode")

Failed to initialize pynvml: NVML Shared Library Not Found
No CUDA-capable GPUs detected


No GPUs available - running in CPU mode


## 2. Load Model for Quick Experimentation

In [5]:
# Create BERT model
model = create_model(
    model_name="bert-base-uncased",
    num_labels=2,
    task_type="classification"
)

# Get model info
info = model.get_model_info()
print(f"Model loaded: {info['model_class']}")
print(f"Parameters: {info['trainable_parameters']:,}")
print(f"Device: {info['device']}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded: BertForSequenceClassification
Parameters: 109,483,778
Device: cpu


## 3. Quick Inference Test

In [6]:
# Test predictions
test_texts = [
    "This movie is absolutely fantastic!",
    "Terrible waste of time.",
    "An okay film with some good moments."
]

predictions = model.predict(test_texts)

for text, pred in zip(test_texts, predictions):
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"{sentiment}: {text}")

Positive: This movie is absolutely fantastic!
Positive: Terrible waste of time.
Positive: An okay film with some good moments.
